In [1]:
%load_ext autoreload
%autoreload 2

from naludaq.board import Board, startup_board
import numpy as np
import time

from naludaq.tools.pedestals import get_pedestals_controller
from naludaq.controllers import get_dac_controller, get_board_controller, get_gainstage_controller, get_readout_controller
from naludaq.parsers import get_parser
from naludaq.communication.analog_registers import AnalogRegisters
from naludaq.tools.waiter import EventWaiter
from naludaq.daq import DebugDaq

from oleas.helpers import (
    readout,
)

### Function Definition

In [2]:
def read_events(board, daq, amount):
    bc = get_board_controller(board)
    buffer = daq.output_buffer
    output = []

    for _ in range(amount):
        bc.toggle_trigger()

        for _ in range(5):
            try:
                waiter = EventWaiter(buffer, amount=1, timeout=1, interval=0.005)
                waiter.start(blocking=True)
                output.append(buffer.popleft())
            except (TimeoutError, IndexError):
                print('Failed to get event, trying again...')
                bc.toggle_trigger()
                continue
            else:
                break
        else:
            print('Maximum number of attempts reached. Aborting.')
    return output

### Connect Board

In [3]:
SERIAL = "210357B4B30CB" # CHANGE ME

board = Board("aodsoc_aods")
board.load_registers()
board.get_ftdi_connection(serial_number=SERIAL, baud=3000000)
startup_board(board)
dc = get_dac_controller(board)
gsc = get_gainstage_controller(board)
ar = AnalogRegisters(board)

gsc.ch0_external_input()
gsc.ch1_8x_ch0()
gsc.ch2_8x_ch1()

READ_WINDOW = {
    'windows': 40,
    'lookback': 40,
    'write_after_trig': 20,
}

Trying [115200, 1000000, 2000000, 3000000]


Try to keep isel between 2000 - 3000
DAC Value can be between 0 - 4095

The provided values should be a good starting point for tuning the board parameters

The DAC value has a larger impact to CH2 compared to CH0, so try adjusting the DAC value to first

get CH2 to about 2048, then start changing ISEL & DAC value to get both CH0 and CH2 close to 2048

In [23]:
# Run this code block modifying the DAC_VALUE and ISEL values to tune CH2 and CH0 avg to 2048.

DAC_VALUE = 2098 # Try to adjust in steps of 5 at first
ISEL = 2652 

dc.set_dacs(DAC_VALUE)
ar.write('isel', ISEL)

with readout(board, READ_WINDOW) as daq:
    events = read_events(board, daq, amount=5)
print(f"Channel 0 Avg: {np.mean([event['data'][0] for event in events])}")
print(f"Channel 1 Avg: {np.mean([event['data'][1] for event in events])}")
print(f"Channel 2 Avg: {np.mean([event['data'][2] for event in events])}")

Channel 0 Avg: 2017.678671875
Channel 1 Avg: 2086.238671875
Channel 2 Avg: 2025.6446875


In [ ]:
board.disconnect()